This demo is designed to work with the standard socket workers alice and bob from the other demos. It adds two numbers together

In [1]:
import syft as sy
import torch
from syft.spdz import spdz
from syft.core.frameworks.torch import _GeneralizedPointerTensor as gpt
from syft.core.frameworks.torch import _SPDZTensor as spdzt

Utility fuction to print chains

In [2]:
from syft.core.frameworks.torch.utils import chain_print as pp

In [3]:
hook = sy.TorchHook(verbose=False)
me = hook.local_worker
bob = sy.VirtualWorker(id="bob",hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice",hook=hook, is_client_worker=False)
me.is_client_worker = False

compute_nodes = [bob, alice]

me.add_workers([bob, alice])
bob.add_workers([me, alice])
alice.add_workers([me, bob])

In [4]:
x = torch.LongTensor([3])
y = torch.LongTensor([5])

In [5]:
x_enc = spdz.encode(x)
y_enc = spdz.encode(y)

In [6]:
x_alice, x_bob = spdz.share(x_enc)
y_alice, y_bob = spdz.share(y_enc)

In [7]:
x_alice.send(alice)
x_bob.send(bob)
y_alice.send(alice)
y_bob.send(bob)

LongTensor[_PointerTensor - id:5989216192 owner:0 loc:bob id@loc:38641853002]

### GPT

In [8]:
x_pointer_tensor_dict = {alice:x_alice.child, bob:x_bob.child}
y_pointer_tensor_dict = {alice:y_alice.child, bob:y_bob.child}

In [9]:
x_gp = gpt(x_pointer_tensor_dict).on(x)
y_gp = gpt(y_pointer_tensor_dict).on(y)

### Towards SPDZT

In [10]:
x_spdz = spdzt(x_gp)
y_spdz = spdzt(y_gp)

In [11]:
sum_spdz = x_spdz + y_spdz

In [12]:
sum_spdz.get()


 8
[syft.core.frameworks.torch.tensor.LongTensor of size 1]

In [13]:
mul_spdz = x_spdz * y_spdz

In [14]:
mul_spdz.get()


 15
[syft.core.frameworks.torch.tensor.LongTensor of size 1]